<a href="https://colab.research.google.com/github/KrishnaPothula/stock_market_analysis/blob/main/hist_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

# Define the stock symbol
symbol = 'WIPRO.NS'
print(symbol)
# Fetch data from Yahoo Finance
df = yf.download(symbol)

# Create a column for quarters
df['Quarter'] = df.index.to_period('Q')

# Group by quarter and calculate the difference between the last and first day of the quarter
quarterly_diff = df.groupby('Quarter')['Close'].apply(lambda x: x.iloc[-1] - x.iloc[0])

# Count the number of positive and negative values for each quarter
quarterly_counts = quarterly_diff.groupby(quarterly_diff.index.quarter).agg(['count', lambda x: (x > 0).sum(), lambda x: (x < 0).sum()])
quarterly_counts.columns = ['Total', 'Positive', 'Negative']

# Calculate the percentage of positive values for each quarter
quarterly_counts['Positive Percentage'] = quarterly_counts['Positive'] / quarterly_counts['Total']

# Create a column for the output based on the percentage of positive values
conditions = [
    (quarterly_counts['Positive Percentage'] > 0.67),
    (quarterly_counts['Positive Percentage'] <= 0.67) & (quarterly_counts['Positive Percentage'] > 0.51),
    (quarterly_counts['Positive Percentage'] <= 0.41),
    (quarterly_counts['Positive Percentage'] > 0.41) & (quarterly_counts['Positive Percentage'] <= 0.51)
]
choices = ['Strong Buy', 'Weak Buy', 'Strong Sell', 'Weak Sell']
quarterly_counts['Output'] = np.select(conditions, choices)

# Print the final output
for index, row in quarterly_counts.iterrows():
    print(f"Q{index}: {row['Output']} ({row['Positive']}, {row['Negative']})")


WIPRO.NS
[*********************100%***********************]  1 of 1 completed
Q1: Weak Buy (15, 13)
Q2: Weak Sell (14, 14)
Q3: Weak Sell (14, 13)
Q4: Strong Buy (19, 8)


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots



# Prompt the user to enter the start and end dates
start_date = input('Enter the start date (YYYY-MM-DD): ')
end_date = input('Enter the end date (YYYY-MM-DD): ')

# Use the yfinance library to retrieve the data for the specified date range
data = yf.download(symbol, start=start_date, end=end_date)

# Create a subplot with 2 rows and 1 column
fig = make_subplots(rows=2, cols=1)

# Add a line chart of the stock's closing price to the first row
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Closing Price'), row=1, col=1)

# Add a candlestick chart to the second row
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'],
                             name='Candlestick'), row=2, col=1)

# Add labels and a title
fig.update_layout(title=f'{symbol} Stock Price',
                  height=800) # Set the height of the figure

# Show the plot
fig.show()


Enter the start date (YYYY-MM-DD): 2022-01-01
Enter the end date (YYYY-MM-DD): 2023-01-01
[*********************100%***********************]  1 of 1 completed
